### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water and 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

### Answer
#### 1.
The least square problem can be fomulated as:

$
\begin{aligned}
\min_{A_{12}, A{21}} \text{loss}  = \sum \left(\hat{p}-p\right)^{2}
\end{aligned}
$

where p can be calculated as:

$
\begin{aligned}
\hat{p} = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}
+ x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}\\
\end{aligned}
$

and saturation pressures are calculated as:
$
\begin{aligned}
p_{1,4 dioxane}^{sat} = 10^{ a_1 - \frac{a_2}{T + a_3}} \\
p_{water}^{sat} = 10^{ a_1 - \frac{a_2}{T + a_3}} \\
\end{aligned}
$

#### 2.
use x2 = 1 - x1

In [4]:
import numpy as np
import torch as t
from torch.autograd import Variable

x1_data = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
p_data = [28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]
p_water = 10.0**(8.07131-1730.63/(20.0+233.426))
p_dio = 10.0**(7.43155-1554.679/(20.0+240.337))

def loss(a): 
    loss_value = 0.0
    for i in range(11):
        x1 = x1_data[i]
        p = p_data[i]
        p_fit = x1*p_water*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*p_dio*t.exp(a[1]*( a[0]*x1/( a[0]*x1+a[1]*(1-x1) ) )**2)
        loss_value = loss_value + (p_fit-p)**2
    return loss_value

err = 1
A = Variable(t.tensor([2.0, 2.0]), requires_grad=True)
# iteration until error < eps
while err >= 0.1:
    loss(A).backward()
    err = t.linalg.norm(A.grad)
    with t.no_grad():       
        A -= 0.01*A.grad
        A.grad.zero_()
    print(err)
    print(A)

tensor(247.0589)
tensor([0.3940, 0.1226], requires_grad=True)
tensor(1055.7034)
tensor([ 2.1655, 10.5299], requires_grad=True)
tensor(176513.8594)
tensor([-1756.4458,  -141.1287], requires_grad=True)
tensor(7.8409)
tensor([-1756.4489,  -141.0503], requires_grad=True)
tensor(7.8473)
tensor([-1756.4519,  -140.9719], requires_grad=True)
tensor(7.8538)
tensor([-1756.4550,  -140.8934], requires_grad=True)
tensor(7.8604)
tensor([-1756.4580,  -140.8149], requires_grad=True)
tensor(7.8669)
tensor([-1756.4611,  -140.7363], requires_grad=True)
tensor(7.8734)
tensor([-1756.4641,  -140.6576], requires_grad=True)
tensor(7.8800)
tensor([-1756.4672,  -140.5789], requires_grad=True)
tensor(7.8866)
tensor([-1756.4702,  -140.5001], requires_grad=True)
tensor(7.8931)
tensor([-1756.4733,  -140.4212], requires_grad=True)
tensor(7.8997)
tensor([-1756.4763,  -140.3423], requires_grad=True)
tensor(7.9063)
tensor([-1756.4794,  -140.2632], requires_grad=True)
tensor(7.9130)
tensor([-1756.4824,  -140.1842], requ

tensor([-1756.8259,  -131.0485], requires_grad=True)
tensor(8.7428)
tensor([-1756.8291,  -130.9611], requires_grad=True)
tensor(8.7512)
tensor([-1756.8323,  -130.8737], requires_grad=True)
tensor(8.7597)
tensor([-1756.8354,  -130.7861], requires_grad=True)
tensor(8.7682)
tensor([-1756.8386,  -130.6985], requires_grad=True)
tensor(8.7767)
tensor([-1756.8418,  -130.6108], requires_grad=True)
tensor(8.7852)
tensor([-1756.8450,  -130.5230], requires_grad=True)
tensor(8.7938)
tensor([-1756.8481,  -130.4351], requires_grad=True)
tensor(8.8024)
tensor([-1756.8513,  -130.3472], requires_grad=True)
tensor(8.8109)
tensor([-1756.8545,  -130.2591], requires_grad=True)
tensor(8.8196)
tensor([-1756.8577,  -130.1710], requires_grad=True)
tensor(8.8282)
tensor([-1756.8608,  -130.0827], requires_grad=True)
tensor(8.8368)
tensor([-1756.8640,  -129.9944], requires_grad=True)
tensor(8.8455)
tensor([-1756.8672,  -129.9060], requires_grad=True)
tensor(8.8542)
tensor([-1756.8704,  -129.8176], requires_grad=T

tensor([-1757.2169,  -119.7991], requires_grad=True)
tensor(9.9336)
tensor([-1757.2202,  -119.6998], requires_grad=True)
tensor(9.9451)
tensor([-1757.2235,  -119.6004], requires_grad=True)
tensor(9.9567)
tensor([-1757.2268,  -119.5009], requires_grad=True)
tensor(9.9682)
tensor([-1757.2301,  -119.4013], requires_grad=True)
tensor(9.9798)
tensor([-1757.2334,  -119.3015], requires_grad=True)
tensor(9.9915)
tensor([-1757.2367,  -119.2016], requires_grad=True)
tensor(10.0031)
tensor([-1757.2400,  -119.1017], requires_grad=True)
tensor(10.0148)
tensor([-1757.2433,  -119.0016], requires_grad=True)
tensor(10.0266)
tensor([-1757.2466,  -118.9014], requires_grad=True)
tensor(10.0384)
tensor([-1757.2499,  -118.8010], requires_grad=True)
tensor(10.0502)
tensor([-1757.2532,  -118.7006], requires_grad=True)
tensor(10.0620)
tensor([-1757.2565,  -118.6000], requires_grad=True)
tensor(10.0739)
tensor([-1757.2598,  -118.4993], requires_grad=True)
tensor(10.0858)
tensor([-1757.2631,  -118.3985], require

tensor([-1757.6306,  -106.6110], requires_grad=True)
tensor(11.6440)
tensor([-1757.6340,  -106.4946], requires_grad=True)
tensor(11.6609)
tensor([-1757.6375,  -106.3781], requires_grad=True)
tensor(11.6779)
tensor([-1757.6409,  -106.2613], requires_grad=True)
tensor(11.6949)
tensor([-1757.6443,  -106.1444], requires_grad=True)
tensor(11.7120)
tensor([-1757.6477,  -106.0274], requires_grad=True)
tensor(11.7291)
tensor([-1757.6511,  -105.9101], requires_grad=True)
tensor(11.7463)
tensor([-1757.6545,  -105.7927], requires_grad=True)
tensor(11.7635)
tensor([-1757.6580,  -105.6751], requires_grad=True)
tensor(11.7808)
tensor([-1757.6614,  -105.5574], requires_grad=True)
tensor(11.7982)
tensor([-1757.6648,  -105.4394], requires_grad=True)
tensor(11.8157)
tensor([-1757.6682,  -105.3213], requires_grad=True)
tensor(11.8332)
tensor([-1757.6716,  -105.2031], requires_grad=True)
tensor(11.8507)
tensor([-1757.6750,  -105.0846], requires_grad=True)
tensor(11.8683)
tensor([-1757.6785,  -104.9660], r

tensor([-1758.1862,   -85.7879], requires_grad=True)
tensor(15.3192)
tensor([-1758.1898,   -85.6347], requires_grad=True)
tensor(15.3520)
tensor([-1758.1935,   -85.4813], requires_grad=True)
tensor(15.3849)
tensor([-1758.1971,   -85.3275], requires_grad=True)
tensor(15.4180)
tensor([-1758.2008,   -85.1733], requires_grad=True)
tensor(15.4513)
tensor([-1758.2045,   -85.0189], requires_grad=True)
tensor(15.4848)
tensor([-1758.2081,   -84.8640], requires_grad=True)
tensor(15.5184)
tensor([-1758.2118,   -84.7089], requires_grad=True)
tensor(15.5522)
tensor([-1758.2155,   -84.5534], requires_grad=True)
tensor(15.5862)
tensor([-1758.2191,   -84.3976], requires_grad=True)
tensor(15.6204)
tensor([-1758.2228,   -84.2414], requires_grad=True)
tensor(15.6547)
tensor([-1758.2264,   -84.0849], requires_grad=True)
tensor(15.6893)
tensor([-1758.2301,   -83.9281], requires_grad=True)
tensor(15.7240)
tensor([-1758.2338,   -83.7709], requires_grad=True)
tensor(15.7589)
tensor([-1758.2374,   -83.6133], r

tensor(20.8979)
tensor([-1758.6283,   -64.6503], requires_grad=True)
tensor(20.9671)
tensor([-1758.6321,   -64.4407], requires_grad=True)
tensor(21.0368)
tensor([-1758.6359,   -64.2303], requires_grad=True)
tensor(21.1071)
tensor([-1758.6396,   -64.0193], requires_grad=True)
tensor(21.1780)
tensor([-1758.6434,   -63.8075], requires_grad=True)
tensor(21.2494)
tensor([-1758.6472,   -63.5951], requires_grad=True)
tensor(21.3214)
tensor([-1758.6510,   -63.3819], requires_grad=True)
tensor(21.3941)
tensor([-1758.6548,   -63.1680], requires_grad=True)
tensor(21.4673)
tensor([-1758.6586,   -62.9533], requires_grad=True)
tensor(21.5411)
tensor([-1758.6624,   -62.7380], requires_grad=True)
tensor(21.6155)
tensor([-1758.6661,   -62.5218], requires_grad=True)
tensor(21.6906)
tensor([-1758.6699,   -62.3050], requires_grad=True)
tensor(21.7662)
tensor([-1758.6737,   -62.0873], requires_grad=True)
tensor(21.8426)
tensor([-1758.6775,   -61.8690], requires_grad=True)
tensor(21.9195)
tensor([-1758.6813

tensor([-1759.0663,   -32.3439], requires_grad=True)
tensor(36.9837)
tensor([-1759.0696,   -31.9741], requires_grad=True)
tensor(37.2487)
tensor([-1759.0729,   -31.6016], requires_grad=True)
tensor(37.5176)
tensor([-1759.0762,   -31.2264], requires_grad=True)
tensor(37.7905)
tensor([-1759.0795,   -30.8485], requires_grad=True)
tensor(38.0674)
tensor([-1759.0828,   -30.4679], requires_grad=True)
tensor(38.3482)
tensor([-1759.0861,   -30.0844], requires_grad=True)
tensor(38.6331)
tensor([-1759.0894,   -29.6981], requires_grad=True)
tensor(38.9220)
tensor([-1759.0925,   -29.3089], requires_grad=True)
tensor(39.2149)
tensor([-1759.0957,   -28.9168], requires_grad=True)
tensor(39.5119)
tensor([-1759.0989,   -28.5217], requires_grad=True)
tensor(39.8130)
tensor([-1759.1021,   -28.1235], requires_grad=True)
tensor(40.1182)
tensor([-1759.1052,   -27.7224], requires_grad=True)
tensor(40.4275)
tensor([-1759.1084,   -27.3181], requires_grad=True)
tensor(40.7411)
tensor([-1759.1115,   -26.9107], r

In [12]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Define a loss
loss = (x[0] - 1)**2 + (x[1] - 2)**2

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
x.grad.numpy()

array([ 0., -4.], dtype=float32)

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).

### Answer

In [2]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Define a loss
loss = (x[0] - 1)**2 + (x[1] - 2)**2

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
x.grad.numpy()

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Let's examine the gradient at a different x.
x.data = t.tensor([2.0, 1.0])
loss = (x[0] - 1)**2 + (x[1] - 2)**2
loss.backward()
x.grad.numpy()

In [ ]:
# Here is a code for gradient descent without line search

import torch as t
from torch.autograd import Variable

x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Fix the step size
a = 0.01

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = (x[0] - 1)**2 + (x[1] - 2)**2
    loss.backward()
    
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        x -= a * x.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.grad.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())